# Data reader-writer (generators)

This notebook is developed to read FES hydrogen supply and divide it by hourly fluctuation and clusters share (refer capacity-splitter).

This notbeook covers hydrogen generation from the following technologies: 
1. Methane Reformation with CCUS
2. Nuclear Electrolysis
3. Biomass gasification
4. Imports
5. Networked Electrolysi
6. Non-networked Electrolysis

Note: 1,2,3,4 - assumed to have steady hydrogen production (no fluctuations).

In [1]:
import os
import sys
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)
os.getcwd()

'C:\\Users\\tatya\\OneDrive - University of Edinburgh\\01 Dissertation\\01 PyPSA\\PyPSA-GB-H2\\notebooks'

In [2]:
import pandas as pd
import numpy as np

In [3]:
# select scenario:
# scenario = 'Consumer Transformation'
# scenario = 'Falling Short'
# scenario = 'Leading the Way'
scenario = 'System Transformation'

In [4]:
# select year:
year = 2050

In [5]:
# select year (to write snapshots):
df = pd.DataFrame()
df['name'] = pd.date_range('2050-01-01 00:00:00','2050-12-31 23:00:00', freq='H')
df['weightings'] = 1
df.to_csv('..\data\LOPF_data\snapshots.csv', index=False)

In [6]:
# read FES-2022:
xls = pd.ExcelFile('..\data\FES2022\FES2022 Workbook V6.xlsx')
df1 = pd.read_excel(xls, 'WS1', header=7)
# get column A name - 'Fuel Type'
col_1 = df1.columns[0]
# get column B name - 'Scenario'
col_2 = df1.columns[1]
# get column D name - 'Category'
col_3 = df1.columns[3]
# get column E name - 'Sector'
col_4 = df1.columns[4]

### Get yearly generation per technology from FES

In [7]:
# get partial dataframe with rows filtered by value in column A & B & D & E - by Scenario & year

# Methane Reformation with CCUS:
df_smr = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Methane Reformation with CCUS']))]
df_smr = df_smr.filter(['Sector', year], axis=1)
df_smr = df_smr.set_index(['Sector'])

# Nuclear Electrolysis:
df_nuclear = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Nuclear Electrolysis']))]
df_nuclear = df_nuclear.filter(['Sector', year], axis=1)
df_nuclear = df_nuclear.set_index(['Sector'])

# Biomass gasification:
df_bio = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Biomass gasification']))]
df_bio = df_bio.filter(['Sector', year], axis=1)
df_bio = df_bio.set_index(['Sector'])

# Non-networked Electrolysis:
df_non = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Capacity') & (df1[col_4].isin(['Non-networked Electrolysis']))]
df_non = df_non.filter(['Sector', year], axis=1)
df_non = df_non.set_index(['Sector'])

# Imports:
df_import = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Imports']))]
df_import = df_import.filter(['Sector', year], axis=1)
df_import = df_import.set_index(['Sector'])

# Networked Electrolysis:
df_net = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Capacity') & (df1[col_4].isin(['Networked Electrolysis']))]
df_net = df_net.filter(['Sector', year], axis=1)
df_net = df_net.set_index(['Sector'])

In [8]:
# read capacity-splitter by clusters:
xls = pd.ExcelFile('..\data\capacity-splitter\capacity-splitter.xlsx')
df2 = pd.read_excel(xls, 'generators-share', header=0)

### 1. Methane Reformation with CCUS

In [9]:
# get hourly generation (& convert TWh to MWh):
df_smr['hourly'] = df_smr.iloc[0][year] / 8760 * 1e6

In [10]:
# split values by Industrial Clusters based on assumed % in capacity-splitter:

df_smr['St Fergus'] = df_smr['hourly'].mul(df2.iloc[0]['St Fergus'])
df_smr['Grangemouth'] = df_smr['hourly'].mul(df2.iloc[0]['Grangemouth'])
df_smr['Teesside'] = df_smr['hourly'].mul(df2.iloc[0]['Teesside'])
df_smr['Humberside'] = df_smr['hourly'].mul(df2.iloc[0]['Humberside'])
df_smr['Theddlethorpe'] = df_smr['hourly'].mul(df2.iloc[0]['Theddlethorpe'])
df_smr['Merseyside'] = df_smr['hourly'].mul(df2.iloc[0]['Merseyside'])
df_smr['Barrow'] = df_smr['hourly'].mul(df2.iloc[0]['Barrow'])
df_smr['Bacton'] = df_smr['hourly'].mul(df2.iloc[0]['Bacton'])
df_smr['Grain LNG'] = df_smr['hourly'].mul(df2.iloc[0]['Grain LNG'])
df_smr['Southampton'] = df_smr['hourly'].mul(df2.iloc[0]['Southampton'])
df_smr['South Wales'] = df_smr['hourly'].mul(df2.iloc[0]['South Wales'])

df_smr = df_smr.drop(['hourly',year], axis=1)

In [11]:
# rename and re-arrange columns:
df_smr = df_smr.transpose()
df_smr['p_min_pu'] = 1
df_smr['type'] = 'Methane Reformation with CCUS'
df_smr.rename(columns={'Methane Reformation with CCUS': 'p_nom'}, inplace=True)
df_smr = df_smr.rename_axis(None, axis=1).rename_axis('bus', axis=0)

In [12]:
df_smr

,p_nom,p_min_pu,type
bus,,,
St Fergus,3555.701509,1,Methane Reformation with CCUS
Grangemouth,3555.701509,1,Methane Reformation with CCUS
Teesside,3555.701509,1,Methane Reformation with CCUS
Humberside,3555.701509,1,Methane Reformation with CCUS
Theddlethorpe,3555.701509,1,Methane Reformation with CCUS
Merseyside,3555.701509,1,Methane Reformation with CCUS
Barrow,0.000000,1,Methane Reformation with CCUS
Bacton,3555.701509,1,Methane Reformation with CCUS
Grain LNG,0.000000,1,Methane Reformation with CCUS


In [13]:
df_smr.to_csv('..\data\generators\working\h2-smr.csv', index=True)

### 2. Nuclear Electrolysis

In [14]:
# get hourly generation (& convert TWh to MWh):
df_nuclear['hourly'] = df_nuclear.iloc[0][year] / 8760 * 1e6

In [15]:
# split values by Industrial Clusters based on assumed % in capacity-splitter:

df_nuclear['St Fergus'] = df_nuclear['hourly'].mul(df2.iloc[1]['St Fergus'])
df_nuclear['Grangemouth'] = df_nuclear['hourly'].mul(df2.iloc[1]['Grangemouth'])
df_nuclear['Teesside'] = df_nuclear['hourly'].mul(df2.iloc[1]['Teesside'])
df_nuclear['Humberside'] = df_nuclear['hourly'].mul(df2.iloc[1]['Humberside'])
df_nuclear['Theddlethorpe'] = df_nuclear['hourly'].mul(df2.iloc[1]['Theddlethorpe'])
df_nuclear['Merseyside'] = df_nuclear['hourly'].mul(df2.iloc[1]['Merseyside'])
df_nuclear['Barrow'] = df_nuclear['hourly'].mul(df2.iloc[1]['Barrow'])
df_nuclear['Bacton'] = df_nuclear['hourly'].mul(df2.iloc[1]['Bacton'])
df_nuclear['Grain LNG'] = df_nuclear['hourly'].mul(df2.iloc[1]['Grain LNG'])
df_nuclear['Southampton'] = df_nuclear['hourly'].mul(df2.iloc[1]['Southampton'])
df_nuclear['South Wales'] = df_nuclear['hourly'].mul(df2.iloc[1]['South Wales'])

df_nuclear = df_nuclear.drop(['hourly',year], axis=1)

In [16]:
# rename and re-arrange columns:
df_nuclear = df_nuclear.transpose()
df_nuclear['p_min_pu'] = 1
df_nuclear['type'] = 'Nuclear Electrolysis'
df_nuclear.rename(columns={'Nuclear Electrolysis': 'p_nom'}, inplace=True)
df_nuclear = df_nuclear.rename_axis(None, axis=1).rename_axis('bus', axis=0)

In [17]:
df_nuclear

,p_nom,p_min_pu,type
bus,,,
St Fergus,0.000000,1,Nuclear Electrolysis
Grangemouth,0.000000,1,Nuclear Electrolysis
Teesside,0.000000,1,Nuclear Electrolysis
Humberside,0.000000,1,Nuclear Electrolysis
Theddlethorpe,0.000000,1,Nuclear Electrolysis
Merseyside,570.776256,1,Nuclear Electrolysis
Barrow,0.000000,1,Nuclear Electrolysis
Bacton,570.776256,1,Nuclear Electrolysis
Grain LNG,570.776256,1,Nuclear Electrolysis


In [18]:
df_nuclear.to_csv('..\data\generators\working\h2-nuclear.csv', index=True)

### 3. Biomass gasification

In [19]:
# get hourly generation (& convert TWh to MWh):
df_bio['hourly'] = df_bio.iloc[0][year] / 8760 * 1e6

In [20]:
# split values by Industrial Clusters based on assumed % in capacity-splitter:

df_bio['St Fergus'] = df_bio['hourly'].mul(df2.iloc[2]['St Fergus'])
df_bio['Grangemouth'] = df_bio['hourly'].mul(df2.iloc[2]['Grangemouth'])
df_bio['Teesside'] = df_bio['hourly'].mul(df2.iloc[2]['Teesside'])
df_bio['Humberside'] = df_bio['hourly'].mul(df2.iloc[2]['Humberside'])
df_bio['Theddlethorpe'] = df_bio['hourly'].mul(df2.iloc[2]['Theddlethorpe'])
df_bio['Merseyside'] = df_bio['hourly'].mul(df2.iloc[2]['Merseyside'])
df_bio['Barrow'] = df_bio['hourly'].mul(df2.iloc[2]['Barrow'])
df_bio['Bacton'] = df_bio['hourly'].mul(df2.iloc[2]['Bacton'])
df_bio['Grain LNG'] = df_bio['hourly'].mul(df2.iloc[2]['Grain LNG'])
df_bio['Southampton'] = df_bio['hourly'].mul(df2.iloc[2]['Southampton'])
df_bio['South Wales'] = df_bio['hourly'].mul(df2.iloc[2]['South Wales'])

df_bio = df_bio.drop(['hourly',year], axis=1)

In [21]:
# rename and re-arrange columns:
df_bio = df_bio.transpose()
df_bio['p_min_pu'] = 1
df_bio['type'] = 'Biomass gasification'
df_bio.rename(columns={'Biomass gasification': 'p_nom'}, inplace=True)
df_bio = df_bio.rename_axis(None, axis=1).rename_axis('bus', axis=0)

In [22]:
df_bio

,p_nom,p_min_pu,type
bus,,,
St Fergus,0.000000,1,Biomass gasification
Grangemouth,0.000000,1,Biomass gasification
Teesside,0.000000,1,Biomass gasification
Humberside,0.000000,1,Biomass gasification
Theddlethorpe,753.424658,1,Biomass gasification
Merseyside,0.000000,1,Biomass gasification
Barrow,0.000000,1,Biomass gasification
Bacton,753.424658,1,Biomass gasification
Grain LNG,753.424658,1,Biomass gasification


In [23]:
df_bio.to_csv('..\data\generators\working\h2-bio.csv', index=True)

### 4. Imports

In [24]:
# get hourly generation (& convert TWh to MWh):
df_import['hourly'] = df_import.iloc[0][year] / 8760 * 1e6

In [25]:
# split values by Industrial Clusters based on assumed % in capacity-splitter:

df_import['St Fergus'] = df_import['hourly'].mul(df2.iloc[5]['St Fergus'])
df_import['Grangemouth'] = df_import['hourly'].mul(df2.iloc[5]['Grangemouth'])
df_import['Teesside'] = df_import['hourly'].mul(df2.iloc[5]['Teesside'])
df_import['Humberside'] = df_import['hourly'].mul(df2.iloc[5]['Humberside'])
df_import['Theddlethorpe'] = df_import['hourly'].mul(df2.iloc[5]['Theddlethorpe'])
df_import['Merseyside'] = df_import['hourly'].mul(df2.iloc[5]['Merseyside'])
df_import['Barrow'] = df_import['hourly'].mul(df2.iloc[5]['Barrow'])
df_import['Bacton'] = df_import['hourly'].mul(df2.iloc[5]['Bacton'])
df_import['Grain LNG'] = df_import['hourly'].mul(df2.iloc[5]['Grain LNG'])
df_import['Southampton'] = df_import['hourly'].mul(df2.iloc[5]['Southampton'])
df_import['South Wales'] = df_import['hourly'].mul(df2.iloc[5]['South Wales'])

df_import = df_import.drop(['hourly',year], axis=1)

In [26]:
# rename and re-arrange columns:
df_import = df_import.transpose()
df_import['p_min_pu'] = 1
df_import['type'] = 'Imports'
df_import.rename(columns={'Imports': 'p_nom'}, inplace=True)
df_import = df_import.rename_axis(None, axis=1).rename_axis('bus', axis=0)

In [27]:
df_import

,p_nom,p_min_pu,type
bus,,,
St Fergus,0.0,1,Imports
Grangemouth,0.0,1,Imports
Teesside,0.0,1,Imports
Humberside,0.0,1,Imports
Theddlethorpe,0.0,1,Imports
Merseyside,0.0,1,Imports
Barrow,0.0,1,Imports
Bacton,0.0,1,Imports
Grain LNG,0.0,1,Imports


In [28]:
df_import.to_csv('..\data\generators\working\h2-import.csv', index=True)

### 5. Networked Electrolysis

In [29]:
# read FES-2022 to extract electolysers location in 2050:
xls = pd.ExcelFile('..\data\FES2022\FES2022 Workbook V6.xlsx')
df3 = pd.read_excel(xls, 'FL.14', usecols="J,K")

In [30]:
# extract data per selected scenario:
index = df3.loc[df3.isin([scenario]).any(axis=1)].index.tolist()[0]
df3 = df3[(index+4):(index+18)]
df3 = df3.transpose()
df3 = df3.rename(columns=df3.iloc[0]).drop(df3.index[0])

In [31]:
# split by clusters:
df3['St Fergus'] = df3['North Scotland']
df3['Grangemouth'] = df3['South Scotland']
df3['Teesside'] = df3['North East England'] + df3['North West England'] 
df3['Humberside'] = df3['East Midlands'] + df3['Yorkshire']    
df3['Theddlethorpe'] = 0
df3['Merseyside'] = df3['North Wales and Merseyside']
df3['Barrow'] = 0 
df3['Bacton'] = df3['East England']
df3['Grain LNG'] = df3['London'] + df3['South East England'] 
df3['Southampton'] = df3['South England']
df3['South Wales-1'] = df3['South Wales'] + df3['South West England'] + df3['West Midlands'] 

df3 = df3.drop(['East England', 'East Midlands', 'London', 'North East England','North Scotland', 'North Wales and Merseyside', 'North West England',
       'South East England', 'South England', 'South Scotland', 'South Wales','South West England', 'West Midlands', 'Yorkshire'], axis=1)

df3.rename(columns={'South Wales-1': 'South Wales'}, inplace=True)

In [32]:
# get production capacity foe each cluster:
df3 = df3.transpose()
df3['share'] = df3['Unnamed: 10'] / df3.values.sum()
df3['p_nom'] = df3['share'].mul(df_net.iloc[0][year] * 1e3)

In [33]:
# rename and re-arrange columns:
df3 = df3.drop(['Unnamed: 10','share'], axis=1)
df3['p_min_pu'] = 0
df3['type'] = 'Networked Electrolysis'
df3.index.names = ['bus']

In [34]:
df3

,p_nom,p_min_pu,type
bus,,,
St Fergus,7030.867956,0,Networked Electrolysis
Grangemouth,4557.072518,0,Networked Electrolysis
Teesside,3546.952192,0,Networked Electrolysis
Humberside,4670.845583,0,Networked Electrolysis
Theddlethorpe,0.0,0,Networked Electrolysis
Merseyside,3422.41681,0,Networked Electrolysis
Barrow,0.0,0,Networked Electrolysis
Bacton,7020.874376,0,Networked Electrolysis
Grain LNG,1157.717816,0,Networked Electrolysis


In [35]:
# check 'Capacity' GW per FES:
df3['p_nom'].values.sum() / 1e3

34.446333070128944

In [36]:
df3.to_csv('..\data\generators\working\h2-networked.csv', index=True)

### 6. Non-networked Electrolysis

In [37]:
# get hourly generation (& convert TWh to MWh):
df4 = df_non
df4['hourly'] = df4.iloc[0][year] * 1e3

In [38]:
# split values by Industrial Clusters based on assumed % in capacity-splitter:

df4['St Fergus'] = df4['hourly'].mul(df2.iloc[3]['St Fergus'])
df4['Grangemouth'] = df4['hourly'].mul(df2.iloc[3]['Grangemouth'])
df4['Teesside'] = df4['hourly'].mul(df2.iloc[3]['Teesside'])
df4['Humberside'] = df4['hourly'].mul(df2.iloc[3]['Humberside'])
df4['Theddlethorpe'] = df4['hourly'].mul(df2.iloc[3]['Theddlethorpe'])
df4['Merseyside'] = df4['hourly'].mul(df2.iloc[3]['Merseyside'])
df4['Barrow'] = df4['hourly'].mul(df2.iloc[3]['Barrow'])
df4['Bacton'] = df4['hourly'].mul(df2.iloc[3]['Bacton'])
df4['Grain LNG'] = df4['hourly'].mul(df2.iloc[3]['Grain LNG'])
df4['Southampton'] = df4['hourly'].mul(df2.iloc[3]['Southampton'])
df4['South Wales'] = df4['hourly'].mul(df2.iloc[3]['South Wales'])

df4 = df4.drop(['hourly',year], axis=1)

In [39]:
# rename and re-arrange columns:
df4 = df4.transpose()
df4['p_min_pu'] = 0
df4['type'] = 'Non-networked Electrolysis'
df4.rename(columns={'Non-networked Electrolysis': 'p_nom'}, inplace=True)
df4 = df4.rename_axis(None, axis=1).rename_axis('bus', axis=0)

In [40]:
# check 'Capacity' GW per FES:
df4['p_nom'].values.sum() / 1e3

15.105365280173228

In [41]:
df4.to_csv('..\data\generators\working\h2-non-networked.csv', index=True)

### Merge all clusters in one table and re-arrange data for PyPSA

In [42]:
import glob

In [43]:
path = '..\data\generators\working'

In [44]:
all_files = glob.glob(os.path.join(path, "*.csv"))
df5 = pd.concat((pd.read_csv(f) for f in all_files), axis=0, ignore_index=False)

In [45]:
# add unique name
df5['name'] = df5['type'].astype(str) + ': ' + df5['bus'].astype(str)
# add carrier
df5['carrier']='Hydrogen'
# add p_max_pu
df5['p_max_pu']= 1
# rearrange columns
df5 = df5[['name','carrier','type','bus','p_nom','p_max_pu','p_min_pu']]

In [46]:
df5.to_csv('..\data\LOPF_data\generators.csv', index=False)

## Generators-p_max_pu (Non-networked and Networked Electrolysis)

Electrolysers availability is depdendant on renewable electricity generation paterns. Wind Offshore, Wind Onshore and Soloar Photovoltaic maximum avialable power was extracted from PyPSA-GB model and electrolysers availability prorated accordingly.

1. Networked Electrolysis
2. Non-networked Electrolysis

Note: currenly total Wind Offshore, Wind Onshore was used for electrolysers availability estimation without geospacial resolution, i.e. assuming networked electricity. PV was ignored at current stage as it's availability ranges 10-15%.

In [47]:
# scenario is selected based on earlier input: 
if scenario == 'Consumer Transformation': 
    path = '..\data\PyPSA-GB\electricity-renewables\CT\year-2050'
elif scenario == 'Falling Short': 
    path = '..\data\PyPSA-GB\electricity-renewables\FS\year-2050'
elif scenario == 'Leading the Way':
    path = '..\data\PyPSA-GB\electricity-renewables\LW\year-2050'
elif scenario == 'System Transformation':
    path = '..\data\PyPSA-GB\electricity-renewables\ST\year-2050'

In [48]:
path

'..\\data\\PyPSA-GB\\electricity-renewables\\ST\\year-2050'

In [49]:
# read csv:
df6 = pd.read_csv(path+'\generators-p_max_pu.csv')
df6 = df6.set_index(['name'])
df7 = pd.read_csv('..\data\generators\generators-names.csv')

In [50]:
# get average of wind readings as 'networked' and 'non-networked' availability factor:
df6['Networked'] = df6[[col for col in df6.columns if col.startswith('Wind Offshore')]].mean(axis=1)
df6['Non-networked'] = df6[[col for col in df6.columns if col.startswith('Wind Onshore')]].mean(axis=1)
df6 = df6.filter(['Networked','Non-networked'], axis=1)

In [51]:
# add correction factor: Networked Electrolysis:

df_net_f = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Networked Electrolysis']))]
df_net_f = df_net_f.filter(['Sector', year], axis=1)
df_net_f = df_net_f.set_index(['Sector'])

df_net_f = (df_net_f.values.sum() / (df_net.iloc[0][year] * 8760 / 1e3)) / df6['Networked'].values.mean()
df6['Networked + factor'] = df6['Networked'] * df_net_f

# add correction factor: Non-networked Electrolysis:
df_non_f = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Supply') & (df1[col_4].isin(['Non-networked Electrolysis']))]
df_non_f = df_non_f.filter(['Sector', year], axis=1)
df_non_f = df_non_f.set_index(['Sector'])

df_non_f = (df_non_f.values.sum() / (df_non.iloc[0][year] * 8760 / 1e3)) / df6['Non-networked'].values.mean()
df6['Non-networked + factor'] = df6['Non-networked'] * df_non_f

### 1. Networked Electrolysis

In [52]:
df6['Networked Electrolysis' + ': ' + df7.iloc[0]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[1]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[2]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[3]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[4]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[5]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[6]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[7]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[8]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[9]['name']] = df6['Networked + factor']
df6['Networked Electrolysis' + ': ' + df7.iloc[10]['name']] = df6['Networked + factor']

### 2. Non-networked Electrolysis

In [53]:
df6['Non-networked Electrolysis' + ': ' + df7.iloc[0]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[1]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[2]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[3]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[4]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[5]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[6]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[7]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[8]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[9]['name']] = df6['Non-networked + factor']
df6['Non-networked Electrolysis' + ': ' + df7.iloc[10]['name']] = df6['Non-networked + factor']

In [54]:
df6 = df6.drop(['Networked','Non-networked','Networked + factor','Non-networked + factor'], axis=1)

In [55]:
df6.to_csv('..\data\LOPF_data\generators-p_max_pu.csv', index=True)